# Agen Pemesan Contoh Hotel dan Penerbangan

Solusi ini akan membantu Anda memesan tiket penerbangan dan hotel. Skenarionya adalah perjalanan dari London Heathrow LHR tanggal 20 Februari 2024 ke New York JFK dan kembali tanggal 27 Februari 2025 dengan penerbangan kelas ekonomi menggunakan British Airways saja. Saya ingin menginap di hotel Hilton di New York, tolong berikan biaya untuk penerbangan dan hotel.


# Inisialisasi Layanan Azure AI Agent dan dapatkan informasi konfigurasi dari **.env**

### **.env** 

Buat file .env

**.env** berisi string koneksi Layanan Azure AI Agent, model yang digunakan oleh AOAI, dan layanan API Pencarian Google yang sesuai, ENDPOINT, dll.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nama Penyebaran Model Layanan Azure AI Agent Anda"

[**NOTE**] Anda memerlukan model dengan Batas Tingkat 100.000 (Token per menit) Batas Tingkat 600 (Permintaan per menit)

  Anda dapat memperoleh model di Microsoft Foundry - Model dan Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "String Koneksi Proyek Layanan Azure AI Agent Anda"

  Anda dapat memperoleh string koneksi proyek di ringkasan proyek Anda di Layar Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Kunci API Pencarian SERPAPI Anda"
- **SERPAPI_SEARCH_ENDPOINT** = "Endpoint Pencarian SERPAPI Anda"

Untuk mendapatkan Nama Penyebaran Model dan String Koneksi Proyek Layanan Azure AI Agent, Anda perlu membuat Layanan Azure AI Agent. Disarankan menggunakan [template ini](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) untuk membuatnya langsung （***Catatan:*** Layanan Azure AI Agent saat ini hanya disetel di wilayah terbatas. Disarankan Anda merujuk ke [tautan ini](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) untuk mengatur wilayahnya)

Agen perlu mengakses SERPAPI. Disarankan untuk mendaftar menggunakan [tautan ini](https://serpapi.com/searches). Setelah pendaftaran, Anda dapat memperoleh Kunci API dan ENDPOINT yang unik.


# Setup 

Untuk menjalankan notebook ini, Anda perlu memastikan bahwa Anda telah menginstal pustaka yang diperlukan dengan menjalankan `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Versi Semantic Kernel Anda harus minimal 1.27.2.


Muat pengaturan dan sumber daya file .env Anda, pastikan Anda telah menambahkan kunci dan pengaturan Anda serta membuat file .env lokal.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Masuk ke Azure

Sekarang Anda perlu masuk ke Azure. Buka terminal dan jalankan perintah berikut:

```bash
az login
```

Perintah ini akan meminta Anda memasukkan kredensial Azure Anda, memungkinkan layanan Azure AI Agent berfungsi dengan benar.


# Penjelasan:
Ini adalah variabel yang menyimpan kunci API untuk mengakses layanan API SERP (Search Engine Results Page). Kunci API adalah pengenal unik yang digunakan untuk mengotentikasi permintaan yang terkait dengan akun Anda.

Tujuan: Tujuan dari baris ini adalah untuk menyimpan kunci API dalam variabel agar dapat digunakan untuk mengotentikasi permintaan ke layanan API SERP. Kunci API diperlukan untuk mengakses layanan dan melakukan pencarian.
Cara Mendapatkan Kunci API SERP: Untuk mendapatkan kunci API SERP, ikuti langkah umum berikut di https://serpapi.com (langkah-langkah tepatnya dapat berbeda tergantung pada layanan API SERP spesifik yang Anda gunakan):

Pilih Layanan API SERP: Ada beberapa layanan API SERP yang tersedia, seperti SerpAPI, Google Custom Search JSON API, dan lain-lain. Pilih yang paling sesuai dengan kebutuhan Anda.

Daftar Akun: Kunjungi situs web layanan API SERP yang dipilih dan daftar akun. Anda mungkin perlu memberikan beberapa informasi dasar dan memverifikasi alamat email Anda.

Buat Kunci API: Setelah mendaftar, masuk ke akun Anda dan buka bagian API atau dasbor. Cari opsi untuk membuat atau menghasilkan kunci API baru.
Salin Kunci API ke file .env Anda.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Ini adalah variabel yang menyimpan URL dasar untuk endpoint API SERP. Nama variabel BASE_URL adalah konvensi yang digunakan untuk menunjukkan bahwa URL ini adalah titik awal untuk membuat permintaan API.
'https://serpapi.com/search':

Ini adalah string URL aktual yang diberikan ke variabel BASE_URL. Ini mewakili endpoint untuk melakukan kueri pencarian menggunakan API SERP.

# Purpose:
Tujuan dari baris ini adalah untuk mendefinisikan konstanta yang memegang URL dasar untuk API SERP. URL ini akan digunakan sebagai titik awal untuk membangun permintaan API guna melakukan operasi pencarian.

# Usage:
Dengan mendefinisikan URL dasar dalam sebuah variabel, Anda dapat dengan mudah menggunakannya kembali di seluruh kode Anda kapan pun Anda perlu membuat permintaan ke API SERP. Ini membuat kode Anda lebih mudah dipelihara dan mengurangi risiko kesalahan dari pengkodean URL secara hardcode di banyak tempat. Contoh saat ini adalah https://serpapi.com/search?engine=bing yang menggunakan API pencarian Bing. API yang berbeda dapat dipilih di https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Penjelasan:

Di sinilah kode plugin Anda berada.

Definisi Kelas: `class BookingPlugin`: Mendefinisikan sebuah kelas bernama BookingPlugin yang berisi metode untuk pemesanan hotel dan penerbangan.

Metode Pemesanan Hotel:

- `@kernel_function(description="booking hotel")`: Sebuah dekorator yang mendeskripsikan fungsi sebagai fungsi kernel untuk pemesanan hotel.
- `def booking_hotel(self, query: Annotated[str, "Nama kota"], check_in_date: Annotated[str, "Waktu Check-in Hotel"], check_out_date: Annotated[str, "Waktu Check-out Hotel"]) -> Annotated[str, "Mengembalikan hasil pemesanan informasi hotel"]:`: Mendefinisikan sebuah metode untuk pemesanan hotel dengan parameter dan tipe pengembalian yang dianotasi.

Metode ini membangun sebuah dictionary parameter untuk permintaan pemesanan hotel dan mengirim permintaan GET ke API SERP. Ia memeriksa status respon dan mengembalikan properti hotel jika berhasil, atau None jika permintaan gagal.

Metode Pemesanan Penerbangan:

- `@kernel_function(description="booking flight")`: Sebuah dekorator yang mendeskripsikan fungsi sebagai fungsi kernel untuk pemesanan penerbangan.
- `def booking_flight(self, origin: Annotated[str, "Nama Keberangkatan"], destination: Annotated[str, "Nama Tujuan"], outbound_date: Annotated[str, "Tanggal keberangkatan"], return_date: Annotated[str, "Tanggal kembali"]) -> Annotated[str, "Mengembalikan hasil pemesanan informasi penerbangan"]:`: Mendefinisikan sebuah metode untuk pemesanan penerbangan dengan parameter dan tipe pengembalian yang dianotasi.

Metode ini membangun dictionary parameter untuk permintaan penerbangan pergi dan kembali serta mengirim permintaan GET ke API SERP. Ia memeriksa status respon dan menambahkan informasi penerbangan ke string hasil jika berhasil, atau mencetak pesan kesalahan jika permintaan gagal. Metode mengembalikan string hasil yang berisi informasi penerbangan.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Penjelasan:
Pernyataan Impor: Impor modul yang diperlukan untuk kredensial Azure, agen AI, konten pesan obrolan, peran penulis, dan dekorator fungsi kernel.

Pengelola Konteks Asinkron: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ini mengatur pengelola konteks asinkron untuk menangani kredensial Azure dan membuat klien agen AI.

Nama Agen dan Instruksi:
- `AGENT_NAME = "BookingAgent"`: Mendefinisikan nama agen.
- `AGENT_INSTRUCTIONS = """..."""`: Memberikan instruksi rinci untuk agen tentang cara menangani permintaan pemesanan.

Buat Definisi Agen: `agent_definition = await client.agents.create_agent(...)`: Membuat definisi agen dengan model, nama, dan instruksi yang ditentukan.

Buat AzureAI Agent: `agent = AzureAIAgent(...)`: Membuat agen AzureAI menggunakan klien, definisi agen, dan plugin yang telah ditentukan.

Buat Thread: `thread: AzureAIAgentThread | None = None`: Membuat thread untuk agen. Tidak wajib membuat thread terlebih dahulu - jika nilai `None` diberikan, thread baru akan dibuat selama pemanggilan pertama dan dikembalikan sebagai bagian dari respons.

Input Pengguna: `user_inputs = ["..."]`: Mendefinisikan daftar input pengguna untuk diproses agen.

Dalam blok finally, hapus thread dan agen untuk membersihkan sumber daya.


# Autentikasi

Kelas `DefaultAzureCredential` adalah bagian dari Azure SDK untuk Python. Kelas ini menyediakan cara default untuk melakukan autentikasi dengan layanan Azure. Kelas ini mencoba melakukan autentikasi menggunakan beberapa metode dalam urutan tertentu, seperti variabel lingkungan, managed identity, dan kredensial Azure CLI.

Operasi Asinkron: Modul aio menunjukkan bahwa kelas DefaultAzureCredential mendukung operasi asinkron. Ini berarti Anda dapat menggunakannya dengan asyncio untuk melakukan permintaan autentikasi tanpa pemblokiran.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang sahih. Untuk informasi penting, disarankan menggunakan terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau salah tafsir yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
